<a href="https://colab.research.google.com/github/ritwiks9635/Question-Answering-Model/blob/main/Question_Answering_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **🌜Question Answering Model🌛**

In [ ]:
!unzip /content/sample_data/Jee_Exam_peper.zip

In [4]:
! pip install PyMuPDF

## **Data Preprocessing**

In [5]:
import os
import fitz
from PIL import Image
from io import BytesIO
import re


input_folder = "Jee_Exam_peper"
output_folder = "output_questions"


subjects = ["Chemistry", "Math", "Physics"]
for subject in subjects:
    os.makedirs(os.path.join(output_folder, subject), exist_ok=True)


def save_content(content, images, subject_folder, question_num):

    with open(os.path.join(subject_folder, f"Q{question_num}.txt"), "w") as text_file:
        text_file.write(content)


    for i, img_data in enumerate(images):
        image = Image.open(BytesIO(img_data))
        image.save(os.path.join(subject_folder, f"Q{question_num}_img_{i + 1}.png"))


for pdf_file in os.listdir(input_folder):
    if pdf_file.endswith(".pdf"):

        subject = pdf_file.split("_")[0].capitalize()
        subject_folder = os.path.join(output_folder, subject)


        doc = fitz.open(os.path.join(input_folder, pdf_file))
        question_num = 1
        content = ""
        images = []

        is_question = False

        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]

            for block in blocks:
                if "lines" in block:
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span["text"].strip()


                            if re.match(r"^Q\.\d+", text):
                                if content:
                                    save_content(content, images, subject_folder, question_num)
                                    question_num += 1
                                    content = ""
                                    images = []
                                is_question = True
                                content += text + "\n"


                            elif is_question and re.match(r"^\([A-D]\)", text):
                                content += text + "\n"


                            elif is_question:
                                content += text + "\n"


                if "image" in block:
                    img_data = block["image"]
                    images.append(img_data)


        if content:
            save_content(content, images, subject_folder, question_num)

print("Extraction complete. Questions saved by subject.")

Extraction complete. Questions saved by subject.


## **AI-Powered Question Generator:**

In [ ]:
!pip install transformers

In [21]:
import os
import random
import time
from transformers import pipeline

# Define model
generator = pipeline("text-generation", model="distilgpt2")


output_folder = "/content/output_questions"
subjects = ["Chemistry", "Math", "Physics"]

def load_questions(subject):
    subject_folder = os.path.join(output_folder, subject)
    questions = []
    for filename in sorted(os.listdir(subject_folder)):
        if filename.endswith(".txt"):
            with open(os.path.join(subject_folder, filename), "r") as f:
                content = f.read()
                questions.append(content)
    return questions

# Function to generate question base on
def generate_mcq(question_text, difficulty="medium"):
    prompt = (f"Read the following text carefully. If it looks like a question with multiple options, keep it as is. "
              f"If it does not look like a question, generate a new {difficulty} level MCQ question based on the text.\n"
              f"Text: {question_text}")
    generated_question = generator(prompt, max_new_tokens=30, num_return_sequences=1)[0]["generated_text"]
    return generated_question

def ask_questions(subject, difficulty="medium"):
    questions = load_questions(subject)
    question_index = 0

    while question_index < len(questions):
        question_text = questions[question_index]
        start_time = time.time()

        question = generate_mcq(question_text, difficulty)

        print(f"\nQuestion {question_index + 1}:\n{question}")

        user_answer = input("Your Answer (A, B, C, D or type 'skip' to skip, 'exit' to Exit): ").upper()
        end_time = time.time()

        if user_answer == "EXIT":
            print("Exiting quiz...")
            break
        elif user_answer == "SKIP":
            print("Skipping to the next question...")
            answer = "Skipped"
        else:
            print("Answer recorded!")
            answer = user_answer

        history.append({
            "question": question,
            "user_answer": answer,
            "time_taken": round(end_time - start_time, 2)
        })

        question_index += 1

    # Display history
    print("\n--- Question History ---")
    for i, entry in enumerate(history, start=1):
        print(f"\nQuestion {i}: {entry['question']}")
        print(f"User Answer: {entry['user_answer']}")
        print(f"Time Taken: {entry['time_taken']} seconds")


subject_choice = input("Choose subject (Chemistry, Math, Physics): ").capitalize()
difficulty_level = input("Choose difficulty level (easy, medium, hard): ").lower()
ask_questions(subject_choice, difficulty_level)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Choose subject (Chemistry, Math, Physics): Math
Choose difficulty level (easy, medium, hard): Easy


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question 1:
Read the following text carefully. If it looks like a question with multiple options, keep it as is. If it does not look like a question, generate a new easy level MCQ question based on the text.
Text: Q.1

Considering only the principal values of the inverse trigonometric functions, the value of

3
2
cos
−1
√
2
2 + π
2
+ 1
4
sin
−1
2√2 π
2 + π
2
+  tan
−1
√2
π



is
__________ .


The only reason I think that no one is saying we are a

preferring to say for all 3 variables, but it's hard to
Your Answer (A, B, C, D or type 'skip' to skip, 'exit' to Exit): Skip


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Skipping to the next question...

Question 2:
Read the following text carefully. If it looks like a question with multiple options, keep it as is. If it does not look like a question, generate a new easy level MCQ question based on the text.
Text: Q.11

Let
𝑃
1
and
𝑃
2
be two planes given by

𝑃
1
:   10𝑥+ 15𝑦+ 12𝑧−60 = 0 ,

𝑃
2
:    −2𝑥+ 5𝑦+ 4𝑧−20 = 0  .


Which of the following straight lines can be an edge of some tetrahedron whose two faces lie on
𝑃
1

and
𝑃
2

?




(A)
𝑥−1
0

=
𝑦−1
0
=
𝑧−1
5




(B)
𝑥−6
−5

=
𝑦
2

=
𝑧
3


(C)
𝑥
−2

=
𝑦−4
5

=
𝑧
4


(D)
𝑥
1

=
𝑦−4
−2

=
𝑧
3






(E)
𝑥
−7
−5
=
𝑧−1
=
𝑧−1

Your Answer (A, B, C, D or type 'skip' to skip, 'exit' to Exit): Exit
Exiting quiz...

--- Question History ---

Question 1: Generate a easy level MCQ based on the text below. Include a question and four answer options labeled (A), (B), (C), and (D):
User Answer: Skipped
Time Taken: 21.39 seconds

Question 2: Read the following text carefully. If it looks like a question with multi

## **Customizable Quizzes**

In [24]:
# Sample metadata structure for questions by subject
question_metadata = {
    "Chemistry": [
        {"id": "Q1", "topic": "Organic Chemistry", "chapter": "Hydrocarbons", "difficulty": "easy"},
        {"id": "Q2", "topic": "Inorganic Chemistry", "chapter": "Coordination Compounds", "difficulty": "medium"},
        {"id": "Q3", "topic": "Physical Chemistry", "chapter": "Thermodynamics", "difficulty": "hard"},
        {"id": "Q4", "topic": "Analytical Chemistry", "chapter": "Chromatography", "difficulty": "medium"},
        {"id": "Q5", "topic": "Biochemistry", "chapter": "Metabolic Pathways", "difficulty": "hard"},
    ],

    "Math": [
        {"id": "Q1", "topic": "Algebra", "chapter": "Linear Equations", "difficulty": "easy"},
        {"id": "Q2", "topic": "Calculus", "chapter": "Differentiation", "difficulty": "hard"},
        {"id": "Q3", "topic": "Geometry", "chapter": "Circles", "difficulty": "medium"},
        {"id": "Q4", "topic": "Trigonometry", "chapter": "Triangles", "difficulty": "easy"},
        {"id": "Q5", "topic": "Statistics", "chapter": "Probability", "difficulty": "hard"},
    ],

    "Physics": [
        {"id": "Q1", "topic": "Mechanics", "chapter": "Laws of Motion", "difficulty": "medium"},
        {"id": "Q2", "topic": "Optics", "chapter": "Ray Optics", "difficulty": "hard"},
        {"id": "Q3", "topic": "Electromagnetism", "chapter": "Electric Fields", "difficulty": "easy"},
        {"id": "Q4", "topic": "Thermodynamics", "chapter": "Heat Transfer", "difficulty": "medium"},
        {"id": "Q5", "topic": "Waves", "chapter": "Sound Waves", "difficulty": "hard"},
    ]}

output_folder = "output_questions"
os.makedirs(output_folder, exist_ok=True)

def load_question_content(subject, question_id):
    question_path = os.path.join(output_folder, subject, f"{question_id}.txt")
    with open(question_path, "r") as f:
        return f.read()

def filter_questions(subject, topic, chapter, difficulty):
    available_questions = question_metadata.get(subject, [])
    filtered_questions = [
        q for q in available_questions
        if q["topic"] == topic and q["chapter"] == chapter and q["difficulty"] == difficulty
    ]
    return filtered_questions

def create_quiz(subject, topic, chapter, difficulty, num_questions=5):
    filtered_questions = filter_questions(subject, topic, chapter, difficulty)

    if len(filtered_questions) < num_questions:
        print(f"Only {len(filtered_questions)} questions available based on the criteria. Generating quiz with available questions.")
        selected_questions = filtered_questions
    else:
        selected_questions = random.sample(filtered_questions, num_questions)

    # Display the quiz
    print(f"\nCreating a quiz on {subject} - {topic} - {chapter} ({difficulty} level) with {len(selected_questions)} questions.")
    for i, question in enumerate(selected_questions, start=1):
        question_content = load_question_content(subject, question["id"])
        print(f"\nQuestion {i}:\n{question_content}")

def add_question():
    # Allow teacher to add a new question with options and metadata
    subject = input("Enter subject (Chemistry, Math, Physics): ").capitalize()
    if subject not in question_metadata:
        print("Invalid subject.")
        return

    topic = input("Enter topic (e.g., Algebra, Organic Chemistry): ")
    chapter = input("Enter chapter (e.g., Linear Equations, Hydrocarbons): ")
    difficulty = input("Enter difficulty (easy, medium, hard): ").lower()
    if difficulty not in ["easy", "medium", "hard"]:
        print("Invalid difficulty level.")
        return

    question_text = input("Enter the question text: ")
    options = {}
    for option in ["A", "B", "C", "D"]:
        options[option] = input(f"Enter option {option}: ")

    correct_answer = input("Enter the correct option (A, B, C, D): ").upper()
    if correct_answer not in ["A", "B", "C", "D"]:
        print("Invalid correct answer.")
        return


    new_question_id = f"Q{len(question_metadata[subject]) + 1}"
    question_metadata[subject].append({
        "id": new_question_id,
        "topic": topic,
        "chapter": chapter,
        "difficulty": difficulty
    })

    # Save the question and options in a text file
    subject_folder = os.path.join(output_folder, subject)
    os.makedirs(subject_folder, exist_ok=True)
    with open(os.path.join(subject_folder, f"{new_question_id}.txt"), "w") as f:
        f.write(f"{question_text}\n")
        for opt, text in options.items():
            f.write(f"({opt}) {text}\n")
        f.write(f"Correct Answer: {correct_answer}\n")

    print(f"Question {new_question_id} added successfully to {subject}.")

# Interface for teachers to select quiz parameters
def teacher_interface():
    action = input("Choose an action - 'create' to create a quiz or 'add' to add a new question: ").lower()
    if action == "add":
        add_question()
    elif action == "create":
        subject_choice = input("Choose subject (Chemistry, Math, Physics): ").capitalize()
        if subject_choice not in question_metadata:
            print("Invalid subject choice.")
            return

        topics = list(set(q["topic"] for q in question_metadata[subject_choice]))
        topic_choice = input(f"Choose topic {topics}: ")
        if topic_choice not in topics:
            print("Invalid topic choice.")
            return

        chapters = list(set(q["chapter"] for q in question_metadata[subject_choice] if q["topic"] == topic_choice))
        chapter_choice = input(f"Choose chapter {chapters}: ")
        if chapter_choice not in chapters:
            print("Invalid chapter choice.")
            return

        difficulty_level = input("Choose difficulty level (easy, medium, hard): ").lower()
        if difficulty_level not in ["easy", "medium", "hard"]:
            print("Invalid difficulty level.")
            return

        num_questions = int(input("Enter number of questions for the quiz: "))
        create_quiz(subject_choice, topic_choice, chapter_choice, difficulty_level, num_questions)
    else:
        print("Invalid action.")


teacher_interface()

Choose an action - 'create' to create a quiz or 'add' to add a new question: Add
Enter subject (Chemistry, Math, Physics): Math
Enter topic (e.g., Algebra, Organic Chemistry): Algebra
Enter chapter (e.g., Linear Equations, Hydrocarbons): Linear Equations
Enter difficulty (easy, medium, hard): Easy
Enter the question text: What 1 + 1 ?
Enter option A: 1
Enter option B: 2
Enter option C: 3
Enter option D: 4
Enter the correct option (A, B, C, D): B
Question Q6 added successfully to Math.


## **Daily Practice Problems (DPPs):**

In [25]:
from datetime import date

# Sample metadata structure for questions by subject
question_metadata = {
    "Chemistry": [
        {"id": "Q1", "difficulty": "easy"},
        {"id": "Q2", "difficulty": "medium"},
        {"id": "Q3", "difficulty": "hard"},

    ],

    "Math": [
        {"id": "Q1", "difficulty": "easy"},
        {"id": "Q2", "difficulty": "medium"},
        {"id": "Q3", "difficulty": "hard"},

    ],

    "Physics": [
        {"id": "Q1", "difficulty": "easy"},
        {"id": "Q2", "difficulty": "medium"},
        {"id": "Q3", "difficulty": "hard"},

    ]
}

output_folder = "output_questions"
daily_practice_folder = "daily_practice_sets"


os.makedirs(daily_practice_folder, exist_ok=True)

def load_question_content(subject, question_id):
    question_path = os.path.join(output_folder, subject, f"{question_id}.txt")
    with open(question_path, "r") as f:
        return f.read()

def filter_questions_by_difficulty(subject, difficulty):
    return [q for q in question_metadata[subject] if q["difficulty"] == difficulty]

def generate_daily_practice_set(num_questions_per_difficulty=2):
    daily_set = {}

    for subject in question_metadata:
        daily_set[subject] = []

        for difficulty in ["easy", "medium", "hard"]:
            filtered_questions = filter_questions_by_difficulty(subject, difficulty)

            if len(filtered_questions) < num_questions_per_difficulty:
                print(f"Not enough {difficulty} questions for {subject}. Using all available.")
                selected_questions = filtered_questions
            else:
                selected_questions = random.sample(filtered_questions, num_questions_per_difficulty)

            # Load question content and add to daily set
            for question in selected_questions:
                question_content = load_question_content(subject, question["id"])
                daily_set[subject].append({
                    "difficulty": difficulty,
                    "content": question_content
                })

    return daily_set

def save_daily_practice_set(daily_set):
    filename = f"DPP_{date.today()}.txt"
    filepath = os.path.join(daily_practice_folder, filename)

    with open(filepath, "w") as f:
        for subject, questions in daily_set.items():
            f.write(f"\n--- {subject} Daily Practice ---\n")
            for question in questions:
                f.write(f"[{question['difficulty'].capitalize()}]\n{question['content']}\n")

    print(f"Daily Practice Problems saved to {filepath}")


daily_set = generate_daily_practice_set()
save_daily_practice_set(daily_set)

Not enough easy questions for Chemistry. Using all available.
Not enough medium questions for Chemistry. Using all available.
Not enough hard questions for Chemistry. Using all available.
Not enough easy questions for Math. Using all available.
Not enough medium questions for Math. Using all available.
Not enough hard questions for Math. Using all available.
Not enough easy questions for Physics. Using all available.
Not enough medium questions for Physics. Using all available.
Not enough hard questions for Physics. Using all available.
Daily Practice Problems saved to daily_practice_sets/DPP_2024-11-07.txt


##**Performance Dashboard**

In [26]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")


output_folder = "/content/output_questions"
subjects = ["Chemistry", "Math", "Physics"]

# Function to load questions from the output_questions folder
def load_questions(subject):
    subject_folder = os.path.join(output_folder, subject)
    questions = []
    for filename in sorted(os.listdir(subject_folder)):
        if filename.endswith(".txt"):
            with open(os.path.join(subject_folder, filename), "r") as f:
                content = f.read()
                questions.append(content)
    return questions

# Function to generate question and options based on difficulty
def generate_mcq(question_text, difficulty="medium"):
    prompt = (f"Read the following text carefully. If it looks like a question with multiple options, keep it as is. "
              f"If it does not look like a question, generate a new {difficulty} level MCQ question based on the text.\n"
              f"Text: {question_text}")
    generated_question = generator(prompt, max_new_tokens=30, num_return_sequences=1)[0]["generated_text"]
    return generated_question

# Function to ask questions and capture responses with time tracking
def ask_questions(subject, difficulty="medium"):
    questions = load_questions(subject)
    responses = []
    question_index = 0

    while question_index < len(questions):
        question_text = questions[question_index]
        generated_question = generate_mcq(question_text, difficulty)
        print(f"\nQuestion {question_index + 1}:\n{generated_question}")


        start_time = time.time()
        user_answer = input("Your Answer (or type 'skip' to skip and 'exit' to exit): ")
        end_time = time.time()


        time_taken = end_time - start_time

        if user_answer.lower() == "exit":
            print("Exiting quiz...")
            break
        elif user_answer.lower() == "skip":
            print("Skipping to the next question...")
            responses.append({"question": generated_question, "answer": "Skipped", "time_taken": time_taken})
        else:
            print("Answer recorded!")
            responses.append({"question": generated_question, "answer": user_answer, "time_taken": time_taken})

        question_index += 1


    display_dashboard(responses)

# Function to display the performance dashboard
def display_dashboard(responses):
    total_questions = len(responses)
    answered_questions = sum(1 for r in responses if r["answer"] != "Skipped")
    skipped_questions = sum(1 for r in responses if r["answer"] == "Skipped")
    total_time = sum(r["time_taken"] for r in responses)
    avg_time_per_question = total_time / total_questions if total_questions > 0 else 0

    print("\n--- Performance Dashboard ---")
    print(f"Total Questions: {total_questions}")
    print(f"Answered Questions: {answered_questions}")
    print(f"Skipped Questions: {skipped_questions}")
    print(f"Average Time per Question: {avg_time_per_question:.2f} seconds")

    print("\nQuestion-wise Details:")
    for i, response in enumerate(responses, start=1):
        print(f"\nQuestion {i}:")
        print(response["question"])
        print(f"Your Answer: {response['answer']}")
        print(f"Time Taken: {response['time_taken']:.2f} seconds")


subject_choice = input("Choose subject (Chemistry, Math, Physics): ").capitalize()
difficulty_level = input("Choose difficulty level (easy, medium, hard): ").lower()
ask_questions(subject_choice, difficulty_level)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Choose subject (Chemistry, Math, Physics): Math
Choose difficulty level (easy, medium, hard): Easy


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question 1:
Read the following text carefully. If it looks like a question with multiple options, keep it as is. If it does not look like a question, generate a new easy level MCQ question based on the text.
Text: Q.1

Considering only the principal values of the inverse trigonometric functions, the value of

3
2
cos
−1
√
2
2 + π
2
+ 1
4
sin
−1
2√2 π
2 + π
2
+  tan
−1
√2
π



is
__________ .


For example, the simple two-dimensional trigonometric functions are
0 → −1 where
√2 × x = x + 1,
Your Answer (or type 'skip' to skip and 'exit' to exit): B


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Answer recorded!

Question 2:
Read the following text carefully. If it looks like a question with multiple options, keep it as is. If it does not look like a question, generate a new easy level MCQ question based on the text.
Text: Q.11

Let
𝑃
1
and
𝑃
2
be two planes given by

𝑃
1
:   10𝑥+ 15𝑦+ 12𝑧−60 = 0 ,

𝑃
2
:    −2𝑥+ 5𝑦+ 4𝑧−20 = 0  .


Which of the following straight lines can be an edge of some tetrahedron whose two faces lie on
𝑃
1

and
𝑃
2

?




(A)
𝑥−1
0

=
𝑦−1
0
=
𝑧−1
5




(B)
𝑥−6
−5

=
𝑦
2

=
𝑧
3


(C)
𝑥
−2

=
𝑦−4
5

=
𝑧
4


(D)
𝑥
1

=
𝑦−4
−2

=
𝑧
3






(E)
𝑥
−4
=
𝑦−4
−2
=
𝑧−1

Your Answer (or type 'skip' to skip and 'exit' to exit): Exit
Exiting quiz...

--- Performance Dashboard ---
Total Questions: 1
Answered Questions: 1
Skipped Questions: 0
Average Time per Question: 21.71 seconds

Question-wise Details:

Question 1:
Read the following text carefully. If it looks like a question with multiple options, keep it as is. If it does not look like a question, generate a ne

##**User Authentication**

In [29]:
import json
import os
import getpass


credentials_file = "user_credentials.json"

if os.path.exists(credentials_file):
    with open(credentials_file, "r") as file:
        users = json.load(file)
else:
    users = {}

def save_users():
    with open(credentials_file, "w") as file:
        json.dump(users, file)

# Signup function
def signup():
    print("\n--- Signup ---")
    username = input("Enter a username: ")
    if username in users:
        print("Username already exists. Please choose a different username.")
        return False

    password = getpass.getpass("Enter a password: ")
    role = input("Enter your role (student/teacher): ").lower()
    if role not in ["student", "teacher"]:
        print("Invalid role. Please enter 'student' or 'teacher'.")
        return False

    # Save the
    users[username] = {"password": password, "role": role}
    save_users()
    print("Signup successful. You can now log in.")
    return True

# Login function
def login():
    print("\n--- Login ---")
    username = input("Enter your username: ")
    password = getpass.getpass("Enter your password: ")

    user = users.get(username)
    if not user or user["password"] != password:
        print("Invalid username or password.")
        return None

    print(f"Welcome, {username} ({user['role']})!")
    return user["role"]

# Main function to handle authentication and access
def main():
    while True:
        choice = input("\nChoose an action (signup/login/exit): ").lower()
        if choice == "signup":
            signup()
        elif choice == "login":
            role = login()
            if role:
                if role == "teacher":
                    teacher_menu()
                elif role == "student":
                    student_menu()
        elif choice == "exit":
            break
        else:
            print("Invalid choice. Please choose 'signup', 'login', or 'exit'.")

# Teacher menu after successful login
def teacher_menu():
    print("\n--- Teacher Menu ---")
    teacher_interface()

# Student menu after successful login
def student_menu():
    print("\n--- Student Menu ---")
    subject_choice = input("Choose subject (Chemistry, Math, Physics): ").capitalize()
    difficulty_level = input("Choose difficulty level (easy, medium, hard): ").lower()
    ask_questions(subject_choice, difficulty_level)

main()


Choose an action (signup/login/exit): login

--- Login ---
Enter your username: Riju
Enter your password: ··········
Welcome, Riju (student)!

--- Student Menu ---
Choose subject (Chemistry, Math, Physics): Math
Choose difficulty level (easy, medium, hard): Easy


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Question 1:
Read the following text carefully. If it looks like a question with multiple options, keep it as is. If it does not look like a question, generate a new easy level MCQ question based on the text.
Text: Q.1

Considering only the principal values of the inverse trigonometric functions, the value of

3
2
cos
−1
√
2
2 + π
2
+ 1
4
sin
−1
2√2 π
2 + π
2
+  tan
−1
√2
π



is
__________ .


If we assign the x + cos to the pi in
for(+2/3 *cos),
(n + cos) = n
Your Answer (or type 'skip' to skip and 'exit' to exit): Exit
Exiting quiz...

--- Performance Dashboard ---
Total Questions: 0
Answered Questions: 0
Skipped Questions: 0
Average Time per Question: 0.00 seconds

Question-wise Details:

Choose an action (signup/login/exit): Exit


##**AI Chatbot:**

In [30]:
chatbot = pipeline("text-generation", model="distilgpt2")


predefined_responses = {
    "how to create a quiz": "To create a quiz, go to the Teacher Menu and select the 'Create Quiz' option. Then, choose your desired topic, chapter, and difficulty level.",
    "how to add a question": "To add a question, go to the Teacher Menu and select the 'Add Question' option. You'll need to enter the question, answer choices, and correct answer.",
    "study tips": "It's best to review concepts regularly, practice questions, and take breaks to refresh your mind. Use quizzes to test your knowledge.",
    "exam preparation tips": "Organize your study time, focus on weak areas, and practice past exam questions. Quizzes are a great way to get exam-ready."
}


def get_chatbot_response(query):

    for key, response in predefined_responses.items():
        if key in query.lower():
            return response


    prompt = f"User asked: '{query}'\nChatbot response:"
    generated_response = chatbot(prompt, max_new_tokens=50, num_return_sequences=1)[0]["generated_text"]


    response_lines = generated_response.split("\n")
    if len(response_lines) > 1:
        return response_lines[1].strip()
    return generated_response.strip()


def chatbot_interface():
    print("\n--- AI Chatbot ---")
    print("You can ask me anything about the platform or exam preparation.")
    print("Type 'exit' to end the conversation.")

    while True:
        user_query = input("You: ")
        if user_query.lower() == "exit":
            print("Chatbot: Goodbye! If you need further assistance, feel free to ask later.")
            break


        response = get_chatbot_response(user_query)
        print(f"Chatbot: {response}")


chatbot_interface()

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



--- AI Chatbot ---
You can ask me anything about the platform or exam preparation.
Type 'exit' to end the conversation.
You: How to create quizze


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Chatbot: Chatbot response: 'I was a teacher with a teacher. I had a very long, very good working life so I worked hard as a teacher. I think everyone was involved - that was really exciting to me."
You: how to create a quiz
Chatbot: To create a quiz, go to the Teacher Menu and select the 'Create Quiz' option. Then, choose your desired topic, chapter, and difficulty level.
You: Exit
Chatbot: Goodbye! If you need further assistance, feel free to ask later.


## **Integration with Ed-tech Platforms & Feedback System**

This step not completed